In [2]:
%matplotlib inline

import os,sys
import numpy as np
import healpy as hp
import pandas as pd
import gmca4im_lib2 as g4i
import astropy.io.fits as fits
import Extension4BINGO as cs
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
rc('text', usetex=True)
font = {'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

In [3]:
data             = "BINGO"  #MeerKAT
wtransform       = "starlet"
maps_wout_mean   = True
mask             = True
add_noise  = True
sigmaE           = None    #add sigma value. If it is blank will be assume 21cm fluctuations
name_noise = None

#Wavelete informations
J     = 3
use_c = True

################   GMCA PARAMETERS   ##################
n_s   = 3   # number of sources to be estimated
mints = 0.1 # min threshold (what is sparse compared to noise?)
nmax  = 100 # number of iterations (usually 100 is safe)
L0    = 0   # switch between L0 norm (1) or L1 norm (0)
#######################################################
AInit     = None
ColFixed  = None
whitening = False
epsi      = 1e-3
verbose   = False

#GMCAExtension
div =1 #  J+1  #J/div will should be even number

### Foregrounds + HI + Noise
$C_{\ell}^{\textrm{FG+HI+N}}$

In [4]:
#####################################
#Input maps and its names
#####################################
pathmaps  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps"  #Put here your path to the input maps
dir_      = "Cubos_Input_L11_L25_21cm_fg_White_Noise"                               #Put here directory name  of the observacional maps
#####################################
#path Output Cls 
#####################################
pathout = "/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/cls_output" #Put here your path to the output cls
plot_   = "reconstruction" #You should choice between reconstruction or residuals cls values

In [5]:
names     = os.listdir(os.path.join(pathmaps,dir_))
names21cm = []
nseed     = []
for iname in names:
    nseed.append(iname.split("_")[-2]) #selecting L-number from string name
    names21cm.append(os.path.join(dir_,iname)) #stacking maps -- dir/namefile
names21cm = np.asarray(names21cm) 

In [ ]:
timei = time.time()
subdirs = cs.checkdir(pathout, subdirs=["21cm","foregrounds","mixmatrix"])
for i,iname in enumerate(names21cm):
    time0 = time.time()
    maps    = cs.getmaps(type_ = data, path = pathmaps, observed_without_mean = maps_wout_mean, apply_mask=mask,add_noise=add_noise, sigmaE= sigmaE,name_21cm=iname)  #to get map
    X       = maps["observed"]
    mrec    = cs.maps2GMCAmaps(maps=X, WT=wtransform, J=J, div=div, n_s=n_s, mints=mints, nmax=nmax, L=L0, AInit=AInit, ColFixed=ColFixed, whitening=whitening, epsi=epsi, use_scale_coefs=use_c) #Component Separation
    del X, maps
    cs.saveouts(mrec=mrec,pathout=pathout, iseed=nseed[i], J=J, div=div, plot_=plot_,subdirs=subdirs) #save cls in its diretory and with its L-number name
    time0   = time.time()-time0
    print("Loop time: {:.2f} min".format(time0/60))
    del mrec
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))


Wavelet transforming the data . . .
. . completed in 10.06 minutes

Starting Component Analysis...
Building reconstructed maps...
Building residuals maps...
Finished in: 31.85 min
Loop time: 43.63 min


### Noise
$C_{\ell}^{\textrm{N}}$

In [9]:
pathmaps  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps"               #Put here your path to the input map
pathout   = "/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/cls_output" #Put here your path to the output cls
dir_noise = "wn_masked"  #Put here directory name of the noise maps

In [ ]:
names     = os.listdir(os.path.join(pathmaps,dir_noise))
names_noise = []
nseed       = []
for iname in names:
    nseed.append(iname.split("_")[-1][:2]) #selecting L-number from string name
    names_noise.append(os.path.join(dir_noise,iname))
names_noise = np.asarray(names_noise)

In [10]:
timei   = time.time()
subdirs = ["noise"]
if not os.path.isdir(os.path.join(pathout,subdirs[0])):
    os.makedirs(os.path.join(pathout,subdirs[0]))
for i,iname in enumerate(names_noise):
    time0 = time.time()
    maps  = cs.getmaps(type_="noise",  path = pathmaps, name_noise=iname)
    cs.saveouts(mrec=maps, pathout=pathout, iseed="L"+nseed[i], subdirs=subdirs)
    time0 = time.time()-time0
    print("[{0}] Loop time: {1:.2f} min".format("L"+nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 1.84 min


### Prior
$C_{\ell}^{\textrm{prior}}$

In [6]:
pathmaps  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps"               #Put here your path to the input map
pathout   = "/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/cls_output" #Put here your path to the output cls
dir_prior = "Cubos_Prior_WN" #Put here directory name of the noise maps

In [7]:
names       = os.listdir(os.path.join(pathmaps,dir_prior))
names_prior = []
nseed       = []
for iname in names:
    nseed.append(iname.split("_")[-1].split(".")[0][:3]) #selecting L-number from string name
    names_prior.append(os.path.join(dir_prior,iname))
names_prior = np.asarray(names_prior)

In [8]:
timei   = time.time()
subdirs = ["prior"]
if not os.path.isdir(os.path.join(pathout,subdirs[0])):
    os.makedirs(os.path.join(pathout,subdirs[0]))
for i,iname in enumerate(names_prior):
    time0 = time.time()
    maps  = cs.getmaps(type_="prior",  path = pathmaps, name_prior=iname)
    cs.saveouts(mrec=maps, pathout=pathout, iseed=nseed[i], subdirs=subdirs)
    time0 = time.time()-time0
    print("[{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 2.29 min


### 21cm pure
$C_{\ell}^{\textrm{pure}}$

In [3]:
pathmaps  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps"               #Put here your path to the input map
pathout   = "/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/cls_output" #Put here your path to the output cls
dir_prior = "Cubos_21cm_Masked" #Put here directory name of the noise maps

In [4]:
names       = os.listdir(os.path.join(pathmaps,dir_prior))
names_prior = []
nseed       = []
for iname in names:
    nseed.append(iname.split("_")[-1].split(".")[0][:3]) #selecting L-number from string name
    names_prior.append(os.path.join(dir_prior,iname))
names_prior = np.asarray(names_prior)

In [5]:
timei   = time.time()
subdirs = ["pure"]
if not os.path.isdir(os.path.join(pathout,subdirs[0])):
    os.makedirs(os.path.join(pathout,subdirs[0]))
for i,iname in enumerate(names_prior):
    time0 = time.time()
    maps  = cs.getmaps(type_= "pure",  path = pathmaps, name_prior=iname)
    cs.saveouts(mrec=maps, pathout=pathout, iseed=nseed[i], subdirs=subdirs)
    time0 = time.time()-time0
    print("[{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 1.79 min
